## Fase 3.1 — Ingestão de Dados Reais (KaggleHub)

Nesta etapa damos sequência à **Fase 3.1** do plano de atividades, que corresponde à primeira ação prática com dados reais, agora utilizando o **KaggleHub** para baixar o dataset diretamente do repositório oficial do Kaggle.  
Essa abordagem garante que o arquivo seja obtido na versão mais recente, com rastreabilidade e compatibilidade para versionamento com DVC. O processo será executado **dentro do container**, evitando divergências entre ambiente local e remoto.

Após o download, o caminho de saída será exibido para validação, permitindo o rastreamento do arquivo bruto e preparação para push no backend MinIO/DVC.


In [2]:
# ETAPA: DOWNLOAD DOS DADOS REAIS VIA KAGGLEHUB

# Instala o kagglehub dentro do ambiente do notebook
%pip install kagglehub

# Verifica se o pacote foi instalado corretamente
import kagglehub
print("Pacote kagglehub importado com sucesso.")

# Importa biblioteca kagglehub
import kagglehub

# Baixa a versão mais recente do dataset
path = kagglehub.dataset_download("parisrohan/credit-score-classification")

# Exibe caminho de saída para conferência
print("\n Download concluído.")
print("📂 Caminho local do dataset:", path)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 5.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Pacote kagglehub importado com sucesso.


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 9.51M/9.51M [00:01<00:00, 7.39MB/s]

Extracting files...

 Download concluído.
📂 Caminho local do dataset: /root/.cache/kagglehub/datasets/parisrohan/credit-score-classification/versions/1


In [5]:
# ETAPA: MOVER ARQUIVOS USANDO O PATH REAL RETORNADO PELO kagglehub

import os
import shutil

# Use o valor exato retornado pelo kagglehub.dataset_download()
# Se a variável 'path' não existir mais, substitua abaixo pelo caminho real exibido no output
source_dir = path  # exemplo: "/root/.cache/kagglehub/datasets/parisrohan/credit-score-classification/version_123456"

destination_dir = "data/raw"
os.makedirs(destination_dir, exist_ok=True)

# Copia todos os arquivos do diretório real
for filename in os.listdir(source_dir):
    source_file = os.path.join(source_dir, filename)
    destination_file = os.path.join(destination_dir, filename)
    shutil.copy2(source_file, destination_file)

print("Arquivos movidos para:", destination_dir)
print("Conteúdo atual em data/raw/:", os.listdir(destination_dir))


Arquivos movidos para: data/raw
Conteúdo atual em data/raw/: ['test.csv', 'train.csv']


In [6]:
# ETAPA: VERSIONAR ARQUIVOS COM DVC ADD

# Comando shell dentro do notebook Jupyter
# Adiciona todos os arquivos da pasta data/raw/ ao controle DVC
!dvc add data/raw

# Lista o conteúdo para conferir arquivos .dvc criados
!ls -lh data/raw
!cat data/raw.dvc


 ⠋ Checking graph
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/workspace/.dvc/cache/files/md5'| |0/? [00:00<?,    ?fil
                                                                                
!
  0%|          |Adding data/raw to cache              0/2 [00:00<?,     ?file/s]
                                                                                
!
Checking out /workspace/notebooks/data/raw            |0.00 [00:00,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00, 14.93file/s]
total 45M
-rw-r--r-- 1 root root 15M Jul 14 09:17 test.csv
-rw-r--r-- 1 root root 30M Jul 14 09:17 train.csv
outs:
- md5: 5dc4156c8cba026c76905a26de89fdb9.dir
  size: 46502530
  nfiles: 2
  hash: md5
  path: raw


In [7]:
# ETAPA: ENVIAR ARQUIVOS VERSIONADOS PARA BACKEND REMOTO COM DVC PUSH

# Executa push dos dados para o storage remoto configurado
!dvc push

# Verifica status para garantir que tudo foi enviado corretamente
!dvc status


Pushing
!
  0% Querying remote cache|                          |0/1 [00:00<?,    ?files/s]
                                                                                
!
  0% Checking cache in 'mba-mlops-bucket/files/md5'| |0/? [00:00<?,    ?files/s]
                                                                                
!
  0% Checking cache in '/workspace/.dvc/cache/files/md5'| |0/? [00:00<?,    ?fil
                                                                                
!
  0%|          |Pushing to s3                         0/3 [00:00<?,     ?file/s]

!

  0%|          |/workspace/.dvc/cache/files/md50.00/14.7M [00:00<?,        ?B/s]

                                                                                

!

  0%|          |/workspace/.dvc/cache/files/md50.00/29.7M [00:00<?,        ?B/s]

100%|██████████|/workspace/.dvc/cache/file29.7M/29.7M [00:00<00:00,     290MB/s]

                                                                                
1